calcolo numero per ogni classe. --> check class inbalance
uso il codice fornito dall'organizzatore del concorso

## CELL COUNTING
Code copied from:https://github.com/ruchikaverma-iitg/MoNuSAC


This code comes from the supplementary materials of ISBI 2020 MoNuSAC Challenge. (The Verma, Ruchika, et al. "MoNuSAC2020: A Multi-organ Nuclei Segmentation and Classification Challenge." IEEE Transactions on Medical Imaging (2021))

---

Script to count cells (by type) in each image and to store in an excel file with image name in rows and cell-count in columns

#Input

data_path: Specify the path of downloaded images

#Output
An excel file with name 'cell-count.xls' will store on the given output_directory

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
!pip install openslide-bin
import openslide
from xml.dom import minidom
import numpy as np
import openslide
from openslide import open_slide
from glob import glob
import cv2
import matplotlib.pyplot as plt
import scipy.io as sio
from PIL import Image
import scipy
import scipy.ndimage
from shapely.geometry import Polygon
from skimage import draw
!pip install xlwt
import xml.etree.ElementTree as ET
import xlwt
from xlwt import Workbook

In [ ]:
# use the original dataset
data_path = '/content/drive/MyDrive/Colab_Notebooks/PROGETTO/dataset'
os.chdir(data_path)
patients = [x[0] for x in os.walk(data_path)]#Total patients in the data_path
len(patients)


label_map = {'Epithelial':1,
             'Lymphocyte':2,
             'Macrophage':4,
             'Neutrophil':3,
            }

In [ ]:
image_count = 0

# Workbook is created
wb = Workbook()

# add_sheet is used to create sheet.
ccbt = wb.add_sheet('Cell count by type')
ccbt.write(0, 0, 'Image Name') # Add header for image name
ccbt.write(0, 1, 'Epithelial')
ccbt.write(0, 2, 'Lymphocyte')
ccbt.write(0, 3, 'Neutrophil')
ccbt.write(0, 4, 'Macrophage')

total_cell_sums = [0, 0, 0, 0, 0] # Initialize total sums for all cell types (index 0 unused)
# Define the output directory for the Excel file
output_directory = '/content/drive/MyDrive/Colab_Notebooks/PROGETTO'
os.makedirs(output_directory, exist_ok=True)
excel_file_name = os.path.join(output_directory, 'cell-count.xls')

for patient_loc in patients:
    ## Read sub-images of each patient in the data path
    sub_images = glob(patient_loc+'/*.svs')
    for sub_image_loc in sub_images:
        image_count += 1 # Increment image count for current row

        image_name = sub_image_loc
        image_short_name = image_name[len(patient_loc)+1:] # Extract just the image filename
        ccbt.write(image_count, 0, image_short_name) # Add image name in excel file

        # Read xml file
        xml_file_name  = image_name[:-4] + '.xml'
        tree = ET.parse(xml_file_name)
        root = tree.getroot()
        current_image_cell_count = [0,0,0,0,0] # Initialize count for the current image

        # Generate cell count for each type in the current image
        for k in range(len(root)):
            label = None # Initialize label for each annotation
            for child in root[k]:
                for x in child:
                    r = x.tag
                    if r == 'Attribute':
                        label = x.attrib['Name']

                    if r == 'Region':
                        if label in label_map: # Ensure label exists in map before using
                            current_image_cell_count[label_map[label]] += 1
                        # else: # Optional: handle cases where label is not found in label_map
                        #     print(f"Warning: Unknown label '{label}' found in {xml_file_name}")

        # Write current image's cell counts to the excel file
        ccbt.write(image_count, 1, current_image_cell_count[1])
        ccbt.write(image_count, 2, current_image_cell_count[2])
        ccbt.write(image_count, 3, current_image_cell_count[3])
        ccbt.write(image_count, 4, current_image_cell_count[4])

        # Add current image's cell counts to total sums
        for i in range(1, 5): # Summing for indices 1 to 4
            total_cell_sums[i] += current_image_cell_count[i]

# Write the total sums row after all images have been processed
ccbt.write(image_count + 1, 0, 'Total')
ccbt.write(image_count + 1, 1, total_cell_sums[1])
ccbt.write(image_count + 1, 2, total_cell_sums[2])
ccbt.write(image_count + 1, 3, total_cell_sums[3])
ccbt.write(image_count + 1, 4, total_cell_sums[4])

print(f"Final total sums written to Excel: \n Epithelial {total_cell_sums[1]}\n Lymphocyte {total_cell_sums[2]}\n Neutrophil {total_cell_sums[3]}\n Macrophage {total_cell_sums[4]}")
# File stored in the specified output_directory
wb.save(excel_file_name)

Final total sums written to Excel: 
 Epithelial 14539
 Lymphocyte 15654
 Neutrophil 631
 Macrophage 587
